import and install

In [16]:
from pyomo.environ import *
!apt-get install -y glpk-utils



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


Model

In [17]:
d = {1:2, 2:2, 3:3, 4:0, 5:3, 6:1}
Cp = 4
Cs = 2
f  = 100
c  = 200
h  = 50


m = ConcreteModel()

m.T  = RangeSet(1,6)
m.SI = RangeSet(1,7)

m.x = Var(m.T, domain=NonNegativeIntegers)
m.y = Var(m.T, domain=Binary)
m.S = Var(m.SI, domain=NonNegativeIntegers)

# Initial & final inventory
m.init = Constraint(expr=m.S[1] == 1)
m.final = Constraint(expr=m.S[7] == 0)

# Inventory balance
def balance(m,t):
    return m.S[t+1] == m.S[t] + m.x[t] - d[t]
m.balance = Constraint(m.T, rule=balance)



Constraints

In [18]:
# Production capacity
m.cap = Constraint(m.T, rule=lambda m,t: m.x[t] <= Cp*m.y[t])

# Storage capacity
m.store = Constraint(m.SI, rule=lambda m,t: m.S[t] <= Cs)

# Objective (CRITICAL FIX)
m.obj = Objective(
    expr=sum(f*m.y[t] + c*m.x[t] + h*m.S[t] for t in m.T),
    sense=minimize
)


Solve

In [19]:

# ------------------------
# Solve
# ------------------------
SolverFactory("glpk").solve(m)

print("Optimal cost =", value(m.obj))
print("Production:", [int(value(m.x[t])) for t in m.T])


Optimal cost = 2500.0
Production: [1, 2, 3, 0, 4, 0]
